In [1]:
import pandas as pd
import cv2
import glob
import os
import numpy as np

In [2]:
def bbox_convert(row):
    bbox = np.array(row[1:-1].split(", "), dtype=np.float32)
    bbox = bbox.astype(np.int16)
    return bbox

In [3]:
def RGB_extract(image):
    B = 0
    G = 0
    R = 0
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            B += image[x, y, 0]
            G += image[x, y, 1]
            R += image[x, y, 2]
    pixels = image.shape[0] * image.shape[1]
    return f"{int(B/pixels)}, {int(G/pixels)}, {int(R/pixels)}"

In [4]:
labels = pd.read_csv("Fking_final_labels.csv")

In [17]:
row = labels[labels.file_name == "100689967.jpg"]

In [23]:
row['file_name'].values[0]

'100689967.jpg'

In [31]:
filename = row['file_name'].values[0]
image = cv2.imread(os.path.join("./mnt/md0/projects/sami-hackathon/private/data/", filename))
bbox = bbox_convert(row['bbox'].values[0])
face_mean = image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 0, 0), -1)
padding_mean = image[bbox[1] - 50 if bbox[1] > 50 else 0 :bbox[1] + bbox[3] + 50 if bbox[1] + bbox[3] < image.shape[0] else 0, bbox[0] - 50 if bbox[0] > 50 else 0 : bbox[0] + bbox[2] + 50 if bbox[0] + bbox[2] < image.shape[1] else 0]

In [32]:
padding_mean

array([[[ 18,  28,  45],
        [ 34,  42,  55],
        [ 50,  53,  57],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 32,  40,  57],
        [ 75,  82,  99],
        [ 55,  58,  63],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 62,  70,  87],
        [ 89,  97, 110],
        [ 49,  52,  57],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  8,  22,  64],
        [ 10,  23,  67],
        [ 14,  29,  75],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 10,  23,  67],
        [  9,  22,  66],
        [ 10,  23,  69],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  9,  22,  66],
        [  8,  21,  65],
        [  8,  21,  65],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [33]:
cv2.imwrite("pic.jpg", padding_mean)

True

In [8]:
RGB_extract(padding_mean)

'29, 37, 44'

In [42]:
def addRGBMeantodf(row):

    filename = row['file_name']
    image = cv2.imread(os.path.join("./mnt/md0/projects/sami-hackathon/private/data/", filename))
    bbox = bbox_convert(row['bbox'])
    face_mean = image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
    padding_mean = image[bbox[1] - 50 if bbox[1] > 50 else 0 :bbox[1] + bbox[3] + 50 if bbox[1] + bbox[3] < image.shape[0] else 0, bbox[0] - 50 if bbox[0] > 50 else 0 : bbox[0] + bbox[2] + 50 if bbox[0] + bbox[2] < image.shape[1] else 0]

    face_mean = RGB_extract(face_mean)
    cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 0, 0), -1)
    padding_mean = RGB_extract(padding_mean)
    return face_mean, padding_mean


In [43]:
labels["RGB Mean"] = labels.apply(lambda row: addRGBMeantodf(row)[0], axis=1)

1145376.jpg
62228563.jpg
79754536.jpg
996439.jpg


In [44]:
labels["Regional RGB Mean"] = labels.apply(lambda row: addRGBMeantodf(row)[1], axis=1)

1145376.jpg
62228563.jpg
79754536.jpg
996439.jpg


In [45]:
labels.head()


,file_name,height,width,bbox,age,race,masked,skintone,emotion,gender,RGB Mean,Regional RGB Mean
0,100013282.jpg,1333,2000,"[934.0000000000097, 144.82228672769534, 238.24...",20-30s,Caucasian,unmasked,mid-light,Neutral,Male,"97, 120, 154","168, 175, 183"
1,100016175.jpg,1333,2000,"[1094.0513571635438, 422.91772295627203, 55.45...",20-30s,Caucasian,unmasked,light,Neutral,Male,"58, 71, 94","58, 59, 61"
2,10004189.jpg,2000,1333,"[419.93871061403877, 269.1250391680045, 377.19...",20-30s,Mongoloid,unmasked,light,Happiness,Female,"150, 157, 200","228, 230, 237"
3,100104575.jpg,1333,2000,"[1490.6909678848915, 676.0000000000097, 37.553...",20-30s,Caucasian,unmasked,mid-light,Neutral,Male,"137, 153, 176","201, 203, 205"
4,100104600.jpg,2000,1333,"[549.169724453414, 92.52040334013152, 306.8821...",20-30s,Caucasian,unmasked,mid-light,Happiness,Female,"106, 117, 137","180, 182, 185"


In [46]:
labels.to_csv("Fking_final_labels.csv")

In [27]:
for index, row in labels.iterrows():
    filename = row['file_name']
    image = cv2.imread(os.path.join("./mnt/md0/projects/sami-hackathon/private/data/", filename))
    bbox = bbox_convert(row['bbox'])
    face_mean = image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
    (h, w) = face_mean.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    topLeft = face_mean[0:cY, 0:cX]
    topRight = face_mean[0:cY, cX:w]
    bottomLeft = face_mean[cY:h, 0:cX]
    bottomRight = face_mean[cY:h, cX:w]
    cv2.imwrite(f"topLeft.jpg", topLeft)
    cv2.imwrite(f"topRight.jpg", topRight)
    cv2.imwrite(f"bottomLeft.jpg", bottomLeft)
    cv2.imwrite(f"bottomRight.jpg", bottomRight)
    break
    

In [12]:
row = labels.iloc[0]

In [28]:
def four_region_mean(row):
    filename = row['file_name']
    image = cv2.imread(os.path.join("./mnt/md0/projects/sami-hackathon/private/data/", filename))
    bbox = bbox_convert(row['bbox'])
    face_mean = image[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
    (h, w) = face_mean.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    topLeft = face_mean[0:cY, 0:cX] #1
    topRight = face_mean[0:cY, cX:w] #2
    bottomLeft = face_mean[cY:h, 0:cX] #3  
    bottomRight = face_mean[cY:h, cX:w] #4
    part1 = RGB_extract(topLeft)
    part2 = RGB_extract(topRight)
    part3 = RGB_extract(bottomLeft)
    part4 = RGB_extract(bottomRight)
    return part1, part2, part3, part4


In [29]:
labels[['region1', 'region2', 'region3', 'region4']] = labels.apply(four_region_mean, axis=1, result_type='expand')

In [30]:
labels.head()

,file_name,height,width,bbox,age,race,masked,skintone,emotion,gender,RGB Mean,Regional RGB Mean,region1,region2,region3,region4
0,100013282.jpg,1333,2000,"[934.0000000000097, 144.82228672769534, 238.24...",20-30s,Caucasian,unmasked,mid-light,Neutral,Male,"97, 120, 154","168, 175, 183","51, 73, 106","139, 160, 188","72, 100, 141","125, 147, 183"
1,100016175.jpg,1333,2000,"[1094.0513571635438, 422.91772295627203, 55.45...",20-30s,Caucasian,unmasked,light,Neutral,Male,"58, 71, 94","58, 59, 61","90, 106, 133","62, 74, 97","59, 75, 102","25, 33, 46"
2,10004189.jpg,2000,1333,"[419.93871061403877, 269.1250391680045, 377.19...",20-30s,Mongoloid,unmasked,light,Happiness,Female,"150, 157, 200","228, 230, 237","68, 81, 133","141, 150, 194","203, 206, 233","188, 190, 238"
3,100104575.jpg,1333,2000,"[1490.6909678848915, 676.0000000000097, 37.553...",20-30s,Caucasian,unmasked,mid-light,Neutral,Male,"137, 153, 176","201, 203, 205","164, 180, 203","71, 89, 118","220, 226, 236","100, 122, 152"
4,100104600.jpg,2000,1333,"[549.169724453414, 92.52040334013152, 306.8821...",20-30s,Caucasian,unmasked,mid-light,Happiness,Female,"106, 117, 137","180, 182, 185","35, 41, 52","157, 169, 190","60, 75, 103","174, 181, 202"


In [31]:
labels.to_csv("Fking_final_labels.csv")

In [38]:
data = pd.read_csv("Fking_final_labels.csv")

In [37]:
data[data['file_name'] == "1145376.jpg"]

,Unnamed: 0,file_name,height,width,bbox,age,race,masked,skintone,emotion,gender,RGB Mean,Regional RGB Mean,region1,region2,region3,region4
458,458,1145376.jpg,2000,1328,"[590.9311740890672, 1195.7831325301188, 245.66...",20-30s,Mongoloid,unmasked,mid-light,Happiness,Female,"171, 191, 185",[],"193, 188, 174","176, 192, 204","170, 219, 193","145, 165, 171"


In [3]:
data.columns

Index(['Unnamed: 0', 'file_name', 'height', 'width', 'bbox', 'age', 'race',
       'masked', 'skintone', 'emotion', 'gender', 'RGB Mean',
       'Regional RGB Mean', 'region1', 'region2', 'region3', 'region4'],
      dtype='object')

In [40]:
data.apply(lambda row: np.array([int(i) for i in row['RGB Mean'].split(", ")], dtype=np.int32) - np.array([int(i) for i in row['Regional RGB Mean'].split(", ")], dtype=np.int32), axis=1)

0          [-71, -55, -29]
1              [0, 12, 33]
2          [-78, -73, -37]
3          [-64, -50, -29]
4          [-74, -65, -48]
               ...        
15301    [-114, -109, -75]
15302    [-112, -106, -73]
15303    [-129, -125, -82]
15304      [-83, -66, -31]
15305      [-94, -79, -42]
Length: 15306, dtype: object

In [44]:
data.loc[15301, 'file_name']

'image_2713.jpg'

: 

In [43]:
cv2.imread("./mnt/md0/projects/sami-hackathon/private/data/100013282.jpg")